In [1]:
import glob
from PIL import Image
import os
import cv2
import numpy as np
from keras.models import Sequential,load_model
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten,Activation,BatchNormalization
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [52]:
save_file_directory='/content/drive/My Drive/Colab Notebooks/해커톤/data/'
poketmon=['꼬부기','이상해씨','파이리','버터플','뮤','푸린','마자용','나옹','디그다','야돈','고라파덕','꼬마돌','쥬레곤','뽀뽀라','또가스','고오스','잉어킹','메타몽','이브이','피카츄','잠만보','롱스톤','치코리타','브케인','리아코','토게피','데덴네','마릴','나몰빼미']
attr=[]
label=[]

def one_hot_encoder(string):
    poketmon=['꼬부기','이상해씨','파이리','버터플','뮤','푸린','마자용','나옹','디그다','야돈','고라파덕','꼬마돌','쥬레곤','뽀뽀라','또가스','고오스','잉어킹','메타몽','이브이','피카츄','잠만보','롱스톤','치코리타','브케인','리아코','토게피','데덴네','마릴','나몰빼미']
    result=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    index = poketmon.index(string)
    result[index]+=1
    return np.array(result)

def detail_one_hot_encoder(n):
    result = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    result[n]+=1
    return np.array(result)

for name in poketmon:
    for index in range(1,21):
        filename = save_file_directory+name+'/'+name+"_"+str(index)+'.jpg'
        img= load_img(filename)
        x=img_to_array(img)
        attr.append(x)
        label.append(one_hot_encoder(name))

attr = np.array(attr)
label = np.array(label)

In [41]:
MODEL_DIR = '/content/drive/My Drive/Colab Notebooks/해커톤/data/models/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
model_path = '/content/drive/My Drive/Colab Notebooks/해커톤/data/models/{epoch:02d} - {val_loss:2f}.hdf5'
checkpointer = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1,save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10)
    
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),input_shape=(150,150,3),activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64,(3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(29,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(attr,label,validation_data=(attr,label),epochs=100,batch_size=145, verbose=1,callbacks=[early_stopping_callback,checkpointer])



Epoch 1/100
4/4 [==============================] - ETA: 0s - loss: 7.0485 - accuracy: 0.0569
Epoch 00001: val_loss improved from inf to 9.08324, saving model to /content/drive/My Drive/Colab Notebooks/해커톤/data/models/01 - 9.083241.hdf5
4/4 [==============================] - 1s 189ms/step - loss: 7.0485 - accuracy: 0.0569 - val_loss: 9.0832 - val_accuracy: 0.0448
Epoch 2/100
4/4 [==============================] - ETA: 0s - loss: 3.2943 - accuracy: 0.0534
Epoch 00002: val_loss improved from 9.08324 to 3.94096, saving model to /content/drive/My Drive/Colab Notebooks/해커톤/data/models/02 - 3.940963.hdf5
4/4 [==============================] - 1s 154ms/step - loss: 3.2943 - accuracy: 0.0534 - val_loss: 3.9410 - val_accuracy: 0.0552
Epoch 3/100
4/4 [==============================] - ETA: 0s - loss: 3.2355 - accuracy: 0.0655
Epoch 00003: val_loss improved from 3.94096 to 3.76481, saving model to /content/drive/My Drive/Colab Notebooks/해커톤/data/models/03 - 3.764807.hdf5
4/4 [=========

In [76]:
for name in poketmon:
    detail_save_file_directory='/content/drive/My Drive/Colab Notebooks/해커톤/data/'
    detail_attr=[]
    detail_label=[]
    for index in range(1,21):
        filename = detail_save_file_directory+name+'/'+name+'_'+str(index)+'.jpg'
        img= load_img(filename)
        x=img_to_array(img)
        detail_attr.append(x)
        detail_label.append(detail_one_hot_encoder(index-1))

    detail_attr = np.array(detail_attr)
    detail_label = np.array(detail_label)
    MODEL_DIR = '/content/drive/My Drive/Colab Notebooks/해커톤/data/detail_models/'
    if not os.path.exists(MODEL_DIR):
        os.mkdir(MODEL_DIR)
    model_path = '/content/drive/My Drive/Colab Notebooks/해커톤/data/detail_models/'+name+'.hdf5'
    checkpointer = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1,save_best_only=True)
    early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10)
        
    model=Sequential()
    model.add(Conv2D(32,kernel_size=(3,3),input_shape=(150,150,3),activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(32,(3,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))


    model.add(Conv2D(64,(3,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))
    model.add(Dense(20,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    history = model.fit(detail_attr,detail_label,validation_data=(detail_attr,detail_label),verbose=1,epochs=100,batch_size=10, callbacks=[early_stopping_callback,checkpointer])



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch 48/100
1/2 [==============>...............] - ETA: 0s - loss: 0.3019 - accuracy: 0.9000
Epoch 00048: val_loss improved from 0.44874 to 0.34488, saving model to /content/drive/My Drive/Colab Notebooks/해커톤/data/detail_models/쥬레곤.hdf5
2/2 [==============================] - 1s 450ms/step - loss: 0.2930 - accuracy: 0.8500 - val_loss: 0.3449 - val_accuracy: 0.9000
Epoch 49/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0327 - accuracy: 1.0000
Epoch 00049: val_loss improved from 0.34488 to 0.27810, saving model to /content/drive/My Drive/Colab Notebooks/해커톤/data/detail_models/쥬레곤.hdf5
2/2 [==============================] - 1s 464ms/step - loss: 0.1290 - accuracy: 0.9500 - val_loss: 0.2781 - val_accuracy: 0.9000
Epoch 50/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0699 - accuracy: 1.0000
Epoch 00050: val_loss improved from 0.27810 to 0.23686, saving model to /content/drive/My Drive/Colab Notebooks/해커톤/data/det

In [83]:
def output(path):
    width = 150
    height = 150
    img_jpg = Image.open(path)
    resize_img = img_jpg.resize((width, height), Image.ANTIALIAS)
    input_data=img_to_array(resize_img)
    model = load_model('/content/drive/My Drive/Colab Notebooks/해커톤/data/models/final_model.hdf5')
    name = poketmon[model.predict_classes(np.array([input_data]))[0]]
    detail_model = load_model('/content/drive/My Drive/Colab Notebooks/해커톤/data/detail_models/'+name+'.hdf5')
    jpg_index = detail_model.predict_classes(np.array([input_data]))
    probability=max(model.predict(np.array([input_data]))[0])
    index=jpg_index[0]
    return name,probability,name+'_'+str(index)+'.jpg'
    

In [84]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/시연/시연_1.jpg')

('파이리', 0.7564337, '파이리_15.jpg')

In [85]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/시연/시연_2.jpg')

('마자용', 0.33021763, '마자용_18.jpg')

In [86]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/시연/시연_3.jpg')

('롱스톤', 0.6026569, '롱스톤_2.jpg')

In [87]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/data/이상해씨/이상해씨_12.jpg')

('이상해씨', 0.9844498, '이상해씨_11.jpg')

In [88]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/data/브케인/브케인_12.jpg')

('브케인', 0.9993136, '브케인_11.jpg')

In [89]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/data/파이리/파이리_12.jpg')

('파이리', 0.9879671, '파이리_11.jpg')

In [90]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/시연/test.jpg')

('디그다', 0.96371734, '디그다_0.jpg')

In [91]:
output('/test2.jpg')

('또가스', 0.42726994, '또가스_13.jpg')

In [68]:
pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.3.0          
astor                         0.8.1          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.2.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.8.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bleach                        3.2.1          
blis                          0.4.

# 데이터 불리기

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# save_file_directory='C:/Users/Administrator/Dropbox/2020해커톤/data/'
# poketmon=['꼬부기','이상해씨','파이리','버터플','뮤','푸린','마자용','나옹','디그다','야돈','고라파덕','꼬마돌','쥬레곤','뽀뽀라','또가스','고오스','잉어킹','메타몽','이브이','피카츄','잠만보','롱스톤','치코리타','브케인','리아코','토게피','데덴네','마릴','나몰빼미','라프라스']
# datagen = ImageDataGenerator(
#         rotation_range=40,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest')

# for name in poketmon:
#     for index in range(1,21):
#         i=0
#         filename = name+'_'+str(index)+'.jpg'
#         img = load_img(save_file_directory+name+'/'+filename)  # PIL 이미지
#         x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
#         x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열
#         for batch in datagen.flow(x, batch_size=1,save_to_dir=save_file_directory+name+'/', save_prefix='꼬부기'+'_'+str(index), save_format='jpeg'):
#             i += 1
#             if i > 20:
#                 break
# # 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
# # 지정된 `preview/` 폴더에 저장합니다.


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Administrator/Dropbox/2020해커톤/data/꼬부기/꼬부기_2.jpg'